# Run a basic Hermes Energy calculation

In this notebook, we'll perform a hermes energy calculation on an externally prepared protein

In [ ]:
import json
import os
import sys
import tarfile

from pdbtools import *
import requests
from datetime import datetime
from pathlib import Path

import tengu

## 0) Setup

In [ ]:
# Define our project information
DESCRIPTION = "tengu-py demo notebook"
TAGS = ["qdx", "tengu-py-v1", "demo", "hermes"]
WORK_DIR = Path.home() / "qdx" / "tengu-py-hermes"
OUT_DIR = WORK_DIR / "runs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Set our inputs
PROTEIN_PDB_PATH = WORK_DIR / "test_P.pdb"

In [ ]:
# Get our client, for calling modules and using the tengu API
client = await tengu.build_provider_with_functions(batch_tags=TAGS)

## 1) Conversion
We want to convert our raw files into qdxfs

In [ ]:
(converted_protein,) = await client.convert("PDB", PROTEIN_PDB_PATH)

print(f"{datetime.now().time()} | Running protein conversion!")

14:53:57.021787 | Running protein conversion!


In [ ]:
# we get a list of conformers back from conversion, one for each model. We want to select the first
(picked_protein,) = await client.pick_conformer(converted_protein, 0)

In [ ]:
# We download the picked protein so that we can check it with local tools
pp_download = await picked_protein.download(filepath=WORK_DIR / "pp.json")

## 2) Fragmentation

In [ ]:
(fragmented_protein, ) = await client.fragment(WORK_DIR / "pp.json", 1, None)

## 3) Quantum energy calculation

In [ ]:
hermes_energy = await client.hermes_energy(
    fragmented_protein,
    None,
    None,
)